# Examples from Interfacing Language, Spatial Perception and Cognition in Type Theory with Records by Simon Dobnik and Robin Cooper

The examples use the module `ttrtypes` and `records`.  We add a simple notion of equivalence between types.  If you have previously judged something to be of one type then it will also be a witness for the other type.  Ultimately, one would want a more general treatment of equivalence between predicates.

In [1]:
from ttrtypes import BType,Pred,PType,Possibility,RecType,Fun,ListType,FunType,Ty,LazyObj
from records import Rec
from utils import show,show_latex


## Section 4

In [2]:
Ind = BType('Ind')
Ind.judge('a')
Ind.judge('b')
print(Ind.query('a'))
print(Ind.query('b'))

True
True


In [3]:
left = Pred('left',[Ind,Ind])
right = Pred('right',[Ind,Ind])
left.learn_witness_fun(lambda args: PType(right,[args[1],args[0]]))
right.learn_witness_fun(lambda args: PType(left,[args[1],args[0]]))
left_a_b = PType(left,['a','b'])
right_b_a = PType(right,['b','a'])
M = Possibility('M')
right_b_a.in_poss(M).judge('s1')
left_a_b.in_poss(M).judge('s2')
print(left_a_b.in_poss(M).query('s1'))
print(right_b_a.in_poss(M).query('s2'))
print(show(M))


True
True

M:
_____________________________________________
right(b, a): [s1, s2]
left(a, b): [s2, s1]
_____________________________________________



In [4]:
left_a_b.in_poss(M).query('s3')

False

In [5]:
T_left = RecType({'x':Ind,'y':Ind, 'e':(Fun('v1',Ind,Fun('v2',Ind,PType(left,['v1','v2']))),['x','y'])})
print(show(T_left))
show_latex(T_left)

{x : Ind, y : Ind, e : (lambda v1:Ind . lambda v2:Ind . left(v1, v2), [x, y])}


<IPython.core.display.Latex object>

In [6]:
r_left = Rec({'x':'a','y':'b','e':'s2'})
print(show(r_left))

{x = a, y = b, e = s2}


In [7]:
print(T_left.in_poss(M).query(r_left))

True


In [8]:
print(show(M))


M:
_____________________________________________
right(b, a): [s1, s2]
left(a, b): [s2, s1]
Ind: [a, b]
_____________________________________________



In [9]:
target = Pred('target',[Ind])
landmark = Pred('landmark',[Ind])
T_relation = RecType({'x':Ind,
                      'y':Ind,
                      'c1':(Fun('v',Ind,PType(target,['v'])),['x']),
                      'c2':(Fun('v',Ind,PType(landmark,['v'])),['y'])})
T_relation_left = T_relation.merge(T_left)
print(show(T_relation_left))

{x : Ind, y : Ind, c1 : (lambda v:Ind . target(v), [x]), c2 : (lambda v:Ind . landmark(v), [y]), e : (lambda v1:Ind . lambda v2:Ind . left(v1, v2), [x, y])}


In [10]:
print(T_relation_left.subtype_of(T_relation))

True


## Figure 8

### 1.

In [11]:
Real = BType('Real')
Real.learn_witness_condition(lambda n: isinstance(n,float))
print(Real.query(0.5))
print(Real.witness_cache)

True
[0.5]


In [12]:
Point = RecType({'x':Real,'y':Real,'z':Real})
PointMap = ListType(Point)
map =[Rec({'x':34.0,'y':24.0,'z':48.0}),Rec({'x':56.0,'y':78.0,'z':114.0}),Rec({'x':44.0,'y':46.5,'z':33.7})]
print(PointMap.query(map))

True


### 2.

In [13]:
chair = Pred('chair',[Ind])
Ppty = FunType(Ind,Ty)
chair_prop = Fun('v',Ind,PType(chair,['v']))
print(Ppty.query(chair_prop))

True


In [14]:
PointMapPpty = RecType({'reg':PointMap,'pfun':Ppty})
ParsedPointMap = ListType(PointMapPpty)  #should really be SetType
ObjectDetector = FunType(PointMap,ParsedPointMap)


def external_fake_object_detector(pmap):
    chairs = [[i,j] for i in pmap for j in pmap if i.y - j.y == 54]
    return [Rec({'reg':pmap,'pfun':chair_prop}) for pmap in chairs]

parsed_map = external_fake_object_detector(map)
print(show(parsed_map))
print(ParsedPointMap.query(parsed_map))

[{reg = [{x = 56.0, y = 78.0, z = 114.0}, {x = 34.0, y = 24.0, z = 48.0}], pfun = lambda v:Ind . chair(v)}]
True


### 3.

In [15]:
location = Pred('location',[Ind,PointMap])
IndividualRec = RecType({'a':Ind,'loc':Ty,'c':Ty})
IndividuationFun = FunType(PointMapPpty,IndividualRec)
ind_fun = Fun('p',PointMapPpty,RecType({'a':Ind,
                                'loc':(Fun('v',Ind,PType(location,['v',LazyObj(['p', '.','reg'])])),['a']),
                        'c':(LazyObj(['p', '.', 'pfun']),['a'])}))
print(show(ind_fun.app(parsed_map[0])))

{a : Ind, loc : (lambda v:Ind . location(v, [{x = 56.0, y = 78.0, z = 114.0}, {x = 34.0, y = 24.0, z = 48.0}]), [a]), c : (lambda v:Ind . chair(v), [a])}


## Figure 9

### (a)

In [16]:
person = Pred('person',[Ind])
organism = Pred('organism',[Ind])
Person = RecType({'x':Ind,'c':(Fun('v',Ind, PType(person,['v'])),['x'])})
Person.in_poss(M)
Topos_pers_organism = Fun('r',Person,PType(organism,[LazyObj(['r','.','x'])]))
r1 = Person.create()
print(show(r1))
print(show(Person.poss))

{x = _a0, c = _e0}

M:
_____________________________________________
right(b, a): [s1, s2]
left(a, b): [s2, s1]
Ind: [a, b, _a0]
person(_a0): [_e0]
_____________________________________________



In [17]:
T_organism = Topos_pers_organism.app(r1)
print(show(T_organism))

organism(_a0)


In [18]:
def Apply_topos_non_specific(topos,arg,m):
    if topos.domain_type.query(arg):
        res = topos.app(arg)
        res.judge_nonspec()
        return res.in_poss(m)
T_organism_M = Apply_topos_non_specific(Topos_pers_organism,r1,M)
print(show(T_organism_M.query_nonspec()))
print(show(M))

True

M:
_____________________________________________
right(b, a): [s1, s2]
left(a, b): [s2, s1]
Ind: [a, b, _a0]
person(_a0): [_e0]
organism(_a0): [_e1]
_____________________________________________



### (b)

In [19]:
Dep_Topos_pers_organism = Fun('v',Ind,Fun('s',PType(person,['v']),PType(organism,['v'])))
def Apply_topos_specific(topos,arg,m):
    if topos.domain_type.in_poss(m).query(arg):
        res = topos.in_poss(m).app(arg)
        res.in_poss(m).judge(arg)
        return res.in_poss(m)
#T1_organism_M = Apply_topos_specific(Dep_Topos_pers_organism.app('_a_{0}'),'_e_{0}',M)
T1_organism_M = Apply_topos_specific(Dep_Topos_pers_organism.app('_a0'),'_e0',M)

#print(show(Dep_Topos_pers_organism.app('_a_{0}').domain_type.in_poss(M).query('_e_{0}')))
print(show(Dep_Topos_pers_organism.app('_a0').domain_type.in_poss(M).query('_e0')))

print(show(T1_organism_M))
#print(show(T1_organism_M.query('_e_{0}')))
print(show(T1_organism_M.query('_e0')))
print(show(M))


True
organism(_a0)
True

M:
_____________________________________________
right(b, a): [s1, s2]
left(a, b): [s2, s1]
Ind: [a, b, _a0]
person(_a0): [_e0]
organism(_a0): [_e1, _e0]
_____________________________________________



## Figure 10

In [20]:
artefact = Pred('artefact',[Ind])
spatial_template_under1 = Pred('st_under1',[PointMap,PointMap])
spatial_template_under2 = Pred('st_under2',[PointMap,PointMap])
under1 = Pred('under1',[Ind,Ind])
under2 = Pred('under2',[Ind,Ind])
PersonInd = RecType({'a':Ind,'reg':PointMap,'c':(Fun('v',Ind,PType(person,['v'])),['a'])})
ArtefactInd = RecType({'a':Ind,'reg':PointMap,'c':(Fun('v',Ind,PType(artefact,['v'])),['a'])})
Under1_sit = RecType({'o1':PersonInd,
                      'o2':ArtefactInd,
                      'st':(Fun('v1',PointMap,
                                Fun('v2',PointMap,PType(spatial_template_under1,['v1','v2']))),
                            ['o1.reg','o2.reg'])})
Under2_sit = RecType({'o1':PersonInd,
                      'o2':ArtefactInd,
                      'st':(Fun('v1',PointMap,
                                Fun('v2',PointMap,PType(spatial_template_under2,['v1','v2']))),
                            ['o1.reg','o2.reg'])})
Topos_under1 = Fun('r',Under1_sit,PType(under1,[LazyObj([LazyObj(['r','.','o1']),'.','a']),
                                                LazyObj([LazyObj(['r','.','o2']),'.','a'])]))
Topos_under2 = Fun('r',Under2_sit,PType(under2,[LazyObj([LazyObj(['r','.','o1']),'.','a']),
                                                LazyObj([LazyObj(['r','.','o2']),'.','a'])]))



In [21]:
M1 = Possibility('M1')
u1 = Under1_sit.in_poss(M1).create()
print(show(u1))
u2 = Under2_sit.in_poss(M1).create()
print(show(u2))
print(show(M1))

{o1 = {a = _a1, reg = _a2, c = _e2}, o2 = {a = _a3, reg = _a4, c = _e3}, st = _e4}
{o1 = {a = _a5, reg = _a6, c = _e5}, o2 = {a = _a7, reg = _a8, c = _e6}, st = _e7}

M1:
_____________________________________________
Ind: [a, b, _a1, _a3, _a5, _a7]
person(_a1): [_e2]
artefact(_a3): [_e3]
st_under1(_a2, _a4): [_e4]
person(_a5): [_e5]
artefact(_a7): [_e6]
st_under2(_a6, _a8): [_e7]
_____________________________________________



In [22]:
print(show(Topos_under1.app(u1)))
print(show(Topos_under2.app(u2)))

under1(_a1, _a3)
under2(_a5, _a7)


## Figure 11

In [23]:
#Objs_Alex_Sam = RecType({'o0':PersonInd,'o1':PersonInd,'o2':ArtefactInd,'o3':ArtefactInd})
#Priv_Alex = RecType({'objs':Objs_Alex_Sam, 'bel':(Fun('v',Ind,
#                                                    RecType({'c_me':SingletonType(RecType,)})))})#